# Install dependencies

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# Clone Github repository of YOLOV5 from ultralutics
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!cd yolov5 & pip install -r requirements.txt


In [ ]:
# Install requirements


### Import libraries

In [ ]:
import os
import time
import cv2
import os 
import numpy as np
from matplotlib import pyplot as plt
import torch
import uuid 
import json
from PIL import Image
from ultralytics import YOLO

# Load Model

In [ ]:
model = YOLO("yolo11s-obb.pt")

In [ ]:
model

In [ ]:
img_path = './bus.jpg' # test on image

In [ ]:
#img = cv2.imread(img_path)
results = model.predict(img_path)  # includes NMS
cv2.imread(np.array(results))


# draw bounding boxes from image and label

In [7]:

def draw_bounding_boxes(image_path, annotation_path, output_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Get image dimensions
    img_height, img_width, _ = image.shape

    # Read the annotation file
    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    # Process each line in the annotation file
    for line in lines:
        # Parse the YOLO format (class_id, center_x, center_y, width, height)
        class_id, center_x, center_y, width, height = map(float, line.strip().split())
        
        # Convert normalized values to pixel coordinates
        x_center = int(center_x * img_width)
        y_center = int(center_y * img_height)
        box_width = int(width * img_width)
        box_height = int(height * img_height)

        # Calculate top-left and bottom-right corners of the bounding box
        x1 = int(x_center - box_width / 2)
        y1 = int(y_center - box_height / 2)
        x2 = int(x_center + box_width / 2)
        y2 = int(y_center + box_height / 2)

        # Draw the bounding box on the image
        cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
        # Optionally, add the class_id as a label
        cv2.putText(image, str(int(class_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the output image with bounding boxes
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")

# Usage
image_path = "./data/images/CA01_01.jpg"
annotation_path = "./bounding_box_labels/CA01_01.txt"
output_path = "./outputs/output_image_2.jpg"
draw_bounding_boxes(image_path, annotation_path, output_path)


Annotated image saved to ./outputs/output_image_2.jpg


In [ ]:
results = model(img)
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.show()

In [ ]:
np.array(results.render()).shape

# Real Time Detections

In [2]:
def videoCapture(model):
    cap = cv2.VideoCapture(0) # capture video from device
    while cap.isOpened():
        ret,frame = cap.read()
        results = model(frame)
        
        cv2.imshow('YOLO',np.squeeze(results.render())) # show image with predicted boxes
    
        if cv2.waitKey(10) & 0xFF == ord('q'): # quit if q is pressed
            break
    
    cap.release()
    cv2.destroyAllWindows()

model = torch.hub.load('ultralytics/yolov5',"custom", path='yolov5/runs/train/exp6/weights/best.pt',force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Saba/.cache\torch\hub\master.zip
YOLOv5  2024-10-28 Python-3.9.20 torch-2.5.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
videoCapture(model)

C:\Users\Saba/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saba/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saba/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saba/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Saba/.cache\torch\hub\ultralytics_yolov5_master\models\comm

# FineTune

## Collect Data

In [ ]:
IMAGES_PATH = os.path.join('data', 'images')
labels = ['fck','help'] #define new classes
number_imgs = 20 # define number of images that should be taken

In [ ]:
cap = cv2.VideoCapture(0)
for label in labels:
    print(f'Collecting images for {label}')
    time.sleep(5)
    for img_num in range(number_imgs):
        print(f"Collecting images for {label}, image number {img_num} ")

        ret, frame = cap.read()
    
        imgname = os.path.join(IMAGES_PATH, label+'.'+ str(uuid.uuid1()) + '.jpg') #assign name to image
        #save image
        cv2.imwrite(imgname,frame) 
        cv2.imshow('Image Collection',frame)
        time.sleep(2)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

In [15]:
# git repository for image labeling software
!git clone https://github.com/HumanSignal/labelImg

Cloning into 'labelImg'...


In [16]:
#install it
!pip install pyqt5 lxml --upgrade
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------- -------------------------------- 1.3/6.9 MB 7.4 MB/s eta 0:00:01
   ------------- -------------------------- 2.4/6.9 MB 6.7 MB/s eta 0:00:01
   ---------------------- ----------------- 3.9/6.9 MB 6.7 MB/s eta 0:00:01
   -------------------------------- ------- 5.5/6.9 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------- ----------------------- 1.6/3.8 MB 8.4 MB/s eta 0:00:01
   --------------------------------- ------ 3.1/3.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 8.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
   - -------------------------------------- 1.6/50.1 MB 9.3 MB/s eta 0:00:06
   -- ------------------------------------- 3.1/50.1 MB 8.0 MB/s eta 0:00:06
   --- -----------------------

#  Train model

In [ ]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 15 --data dataset.yaml --weights yolov5s.pt --workers 2

# Load Model

In [6]:
model = torch.hub.load('ultralytics/yolov5',"custom", path='yolov5/runs/train/exp4/weights/best.pt',force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Saba/.cache\torch\hub\master.zip
YOLOv5  2024-10-27 Python-3.9.20 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [10]:
#test real time
videoCapture(model)

In [ ]:
# Full tutorial
#https://www.youtube.com/watch?v=tFNJGim3FXw
import json


# Code for drawing bounding box with any shape

In [2]:

from PIL import Image, ImageDraw

json_path = r"C:\Users\Saba\Desktop\Python\midv500_data\midv500\02_aut_drvlic_new\ground_truth\CA/CA02_01.json"
image_path = r"C:\Users\Saba\Desktop\Python\vigo\AIMV\jpgs\02_aut_drvlic_new\CA/CA02_01.jpg"


def draw_quadrilateral_cv2(image_path, json_path):
    # Load image with cv2
    image = cv2.imread(image_path)
    
    # Load quadrilateral coordinates from JSON
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    # Draw quadrilateral
    points = np.array(data['quad'], np.int32)
    points = points.reshape((-1, 1, 2))
    color = (0, 255, 0)  # Green color in BGR
    thickness = 2
    cv2.polylines(image, [points], isClosed=True, color=color, thickness=thickness)
    
    # Display image with cv2
    cv2.imshow('Image with Quadrilateral', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Usage:
# For OpenCV
draw_quadrilateral_cv2(image_path, json_path)


# code for converting midv500 coordinates to YOLO format

In [3]:


def convert_quad_to_yolo_format(quad, image_width, image_height, class_id=0):
    # Step 1: Extract min/max coordinates
    min_x = min(point[0] for point in quad)
    max_x = max(point[0] for point in quad)
    min_y = min(point[1] for point in quad)
    max_y = max(point[1] for point in quad)
    
    # Step 2: Calculate center coordinates, width, and height
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    box_width = max_x - min_x
    box_height = max_y - min_y
    
    # Step 3: Normalize coordinates to [0, 1] range
    center_x /= image_width
    center_y /= image_height
    box_width /= image_width
    box_height /= image_height

    
    # Format as YOLO annotation
    return f"{class_id} {center_x} {center_y} {box_width} {box_height}"



### going in every subfolder and every file in root folder and converting every json file into YOLO format

In [4]:
def process_all_json_files_in_folder(folder_path):
    path_to_save = r"C:\Users\Saba\Desktop\Python\vigo\AIMV\bounding_box_labels/"
    image_width,image_height  = 1920, 1080  # Replace with your image dimensions

    # Walk through all subfolders and files in the given folder path
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                json_file_path = os.path.join(root, file)
                print(file)
                
                try:
                    with open(json_file_path, 'r') as file:
                        data = json.load(file)
                        try:
                            quad = data["quad"]
                        except:
                            print("could not find quad key")

                    yolo_format = convert_quad_to_yolo_format(quad,image_height,image_width)
                    
                    new_file = os.path.basename(file.name).replace(".json",".txt")

                    with open(f"{path_to_save}/{new_file}", "w") as f:
                        f.write(yolo_format + "\n")
                        
                except Exception as e:
                    print(e)
                    print(f"could not process the file: {json_file_path}")


process_all_json_files_in_folder(r"C:\Users\Saba\Desktop\Python\midv500_data\midv500")             

02_aut_drvlic_new.json
could not find quad key
local variable 'quad' referenced before assignment
could not process the file: C:\Users\Saba\Desktop\Python\midv500_data\midv500\02_aut_drvlic_new\ground_truth\02_aut_drvlic_new.json
CA02_01.json
CA02_02.json
CA02_03.json
CA02_04.json
CA02_05.json
CA02_06.json
CA02_07.json
CA02_08.json
CA02_09.json
CA02_10.json
CA02_11.json
CA02_12.json
CA02_13.json
CA02_14.json
CA02_15.json
CA02_16.json
CA02_17.json
CA02_18.json
CA02_19.json
CA02_20.json
CA02_21.json
CA02_22.json
CA02_23.json
CA02_24.json
CA02_25.json
CA02_26.json
CA02_27.json
CA02_28.json
CA02_29.json
CA02_30.json
CS02_01.json
CS02_02.json
CS02_03.json
CS02_04.json
CS02_05.json
CS02_06.json
CS02_07.json
CS02_08.json
CS02_09.json
CS02_10.json
CS02_11.json
CS02_12.json
CS02_13.json
CS02_14.json
CS02_15.json
CS02_16.json
CS02_17.json
CS02_18.json
CS02_19.json
CS02_20.json
CS02_21.json
CS02_22.json
CS02_23.json
CS02_24.json
CS02_25.json
CS02_26.json
CS02_27.json
CS02_28.json
CS02_29.json
CS0

In [ ]:

confidence_score_threshold = 0.87


# Initialize variables
cap = None
id_images_batch, id_labels = retrieve_saved_ids()
frame_counter = 0  # Use local variable instead of global

# Initialize the Tkinter window
window = tk.Tk()
window.title("ID Detection")
window.geometry("1200x800")

# Function to handle uploading a photo
def upload_photo():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        print(f"Photo selected: {file_path}")
        frame = cv2.imread(file_path)
    return frame
# Function to handle uploading a video
def upload_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    if file_path:
        print(f"Video selected: {file_path}")
        # Process the selected video here

# Function to handle opening the camera
def open_camera():
    global cap, video_label
    cap = cv2.VideoCapture(0)  # Open the default camera

    # Create a label to display the video feed
    video_label = tk.Label(window)
    video_label.pack()

    # Function to update frame in Tkinter window
    def update_frame():
        global frame_counter, id_images_batch, id_labels

        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            window.after(10, update_frame)  # Schedule next frame update
            return

        # Example YOLO processing logic here
        # Convert frame to RGB and update the Tkinter label
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        video_label.imgtk = imgtk
        video_label.configure(image=imgtk)

        # Schedule the next frame update
        video_label.after(10, update_frame)

    # Start updating frames
    update_frame()

# Define a function to handle window close
def on_closing():
    if cap:
        cap.release()  # Release the camera
    window.destroy()  # Close the Tkinter window

# Main menu for selecting analysis mode
def show_main_menu():
    for widget in window.winfo_children():
        widget.destroy()  # Clear existing widgets

    # Create buttons for the main menu
    btn_photo = tk.Button(window, text="Upload Photo", command=upload_photo, height=2, width=20)
    btn_video = tk.Button(window, text="Upload Video", command=upload_video, height=2, width=20)
    btn_camera = tk.Button(window, text="Open Camera", command=open_camera, height=2, width=20)

    # Pack the buttons
    btn_photo.pack(pady=20)
    btn_video.pack(pady=20)
    btn_camera.pack(pady=20)

# Show the main menu on startup
show_main_menu()

# Bind the window close event
window.protocol("WM_DELETE_WINDOW", on_closing)

# Start the Tkinter main loop
window.mainloop()
